In [35]:
import tensorflow as tf

import pandas as pd

import json
import os
from datetime import date,datetime,timedelta,timezone

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest

print(tf.__version__)

2.11.0


In [36]:
#import functions_framework
# @functions_framework.http
# def predict_incident_severity_by_tf(request):

In [37]:
PROJECT_ID='smart-data-ml'
dataset_id='SMartML'
#PATH_FOLDER_ARTIFACTS="gs://demo-tuned-tf-incident-pongthorn/model_tuned"
PATH_FOLDER_ARTIFACTS="model" 

# client = bigquery.Client(PROJECT_ID)

credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)

predict_from_date=os.environ.get('predict_from_date', '')
all_prediction=os.environ.get('all_prediction', '0')  # 1 is all , 0 is 1 day

print(f"Prediction From = {predict_from_date}")
print(f"All prediction = {all_prediction}")

# predict_from_date='2023-03-01'

# map_sevirity_to_class={'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}

Prediction From = 
All prediction = 0


In [38]:
table_id = f"{PROJECT_ID}.{dataset_id}.new_incident"
predictResult_table_id=f"{PROJECT_ID}.{dataset_id}.new_result_prediction_incident"
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']

In [39]:
mapping_file="incident_severity_to_class.json"
with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)

print(map_sevirity_to_class)

{'Cosmetic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [40]:
# Get today's date
prediction_datetime=datetime.now(timezone.utc)
today_str=prediction_datetime.strftime("%Y-%m-%d")
today=datetime.strptime(today_str,"%Y-%m-%d")
print(prediction_datetime)

2023-07-19 20:09:04.125883+00:00


In [41]:
# Yesterday date
if predict_from_date=='':
 yesterday = today - timedelta(days = 1)
 str_yesterday=yesterday.strftime('%Y-%m-%d')
else:
 str_yesterday=predict_from_date

str_today=today.strftime('%Y-%m-%d')

print(f"Get data between {str_yesterday} to {str_today} to predict sevirity level")

Get data between 2023-07-18 to 2023-07-19 to predict sevirity level


In [42]:
def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 return df

In [43]:
if int(all_prediction)==0:
    sql=f"""
    SELECT *  FROM `{table_id}` 
     WHERE DATE(imported_at) >= '{str_yesterday}' and DATE(imported_at) < '{str_today}' 
     order by imported_at
    """
else:
    sql=f"""
    SELECT *  FROM `{table_id}` 
     order by imported_at
    """

print(sql)


    SELECT *  FROM `smart-data-ml.SMartML.new_incident` 
     WHERE DATE(imported_at) >= '2023-07-18' and DATE(imported_at) < '2023-07-19' 
     order by imported_at
    


In [14]:

#LIMIT 2
dfNewData=load_data_bq(sql)
dfNewData=dfNewData.drop_duplicates(subset=['id'],keep='last')

dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)


print(dfNewData.info())
# print(dfNewData)

if len(dfNewData)==0:
    print("No Data To predict")
    quit()
    #return "No Data To predict"
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         37 non-null     Int64         
 1   severity_id                37 non-null     Int64         
 2   severity                   37 non-null     int64         
 3   severity_name              37 non-null     object        
 4   sla                        37 non-null     object        
 5   product_type               37 non-null     object        
 6   brand                      37 non-null     object        
 7   service_type               37 non-null     object        
 8   incident_type              37 non-null     object        
 9   open_to_close_hour         37 non-null     float64       
 10  response_to_resolved_hour  37 non-null     float64       
 11  imported_at                37 non-null     datetime64[ns]
dtypes: Int64(2

In [15]:
try:
    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    # print(model.summary())
except Exception as error:
    
  print(str(error))
  raise error


Load from model


In [16]:
pdPrediction=pd.DataFrame(columns=['_id','predict_severity','prob_severity'])

for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


      predictionResult = model.predict(input_dict)
      result_str=','.join([ str(prob) for prob in predictionResult[0]])  
      print(result_str)   

      prob = tf.nn.softmax(predictionResult)
      prob_pct=(100 * prob)  
      _class = tf.argmax(predictionResult,-1).numpy()[0]
      
      dictPrediction={'_id':incident_id, 'predict_severity':_class,'prob_severity':result_str} 
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([dictPrediction])] )

      print(f"{prob_pct} %   as {_class}")     
      print("======================================================================================")
            
dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData['predict_severity']=dfPredictData['predict_severity'].astype('int')
dfPredictData=dfPredictData[['id','prob_severity','predict_severity','severity']]
dfPredictData['prediction_item_date']= datetime.strptime(str_yesterday, '%Y-%m-%d')
dfPredictData['prediction_datetime']=prediction_datetime

2856 - 2(Major)
1/1 [==============================] - 0s 421ms/step
0.0812989,0.6865528,0.2217652,0.010383062
[[20.349737 37.274967 23.41869  18.956604]] %   as 1
3171 - 2(Major)
1/1 [==============================] - 0s 44ms/step
0.0062230583,0.07276667,0.8138062,0.10720422
[[18.458546 19.728632 41.39295  20.419874]] %   as 2
3161 - 2(Major)
1/1 [==============================] - 0s 46ms/step
0.010316226,0.14508697,0.79600173,0.048595004
[[18.597021 21.280092 40.800194 19.322691]] %   as 2
3168 - 2(Major)
1/1 [==============================] - 0s 46ms/step
0.0020216804,0.02833116,0.96639663,0.0032505246
[[17.695871 18.167618 46.418877 17.71763 ]] %   as 2
3182 - 2(Major)
1/1 [==============================] - 0s 47ms/step
0.015951023,0.49201888,0.4240149,0.06801524
[[19.353296 31.153622 29.105482 20.3876  ]] %   as 1
3176 - 2(Major)
1/1 [==============================] - 0s 52ms/step
0.0005379665,0.13916841,0.8547235,0.0055700354
[[18.17123  20.873283 42.69258  18.262901]] %   as 2
3

In [17]:
print(dfPredictData.info())
print(dfPredictData)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    37 non-null     object        
 1   prob_severity         37 non-null     object        
 2   predict_severity      37 non-null     int32         
 3   severity              37 non-null     int64         
 4   prediction_item_date  37 non-null     datetime64[ns]
 5   prediction_datetime   37 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(1), object(2)
memory usage: 1.7+ KB
None
      id                                      prob_severity  predict_severity  \
0   2856          0.0812989,0.6865528,0.2217652,0.010383062                 1   
1   3171       0.0062230583,0.07276667,0.8138062,0.10720422                 2   
2   3161      0.010316226,0.14508697,0.79600173,0.048595004                 2   
3   3168    0.0020216804,0

In [23]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prob_severity", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("predict_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity", "INTEGER", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_item_date", "DATETIME", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") 
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.DemoSMartDW.new_result_prediction_incident already exists.


In [24]:
def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            dfPredictData, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(dfPredictData), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  48 Imported bigquery successfully


In [ ]:
# return 'ok'

In [ ]:
# if __name__ == "__main__":
#  result=predict_incident_severity_by_tf(None)
#  print(result)
